In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('../data/exploratoryData.csv')

In [3]:
df.shape

(484506, 21)

# Clean Dates & Time

In [4]:
# SearchDate is the time of search event
# What year and month does the data range from?
import datetime

'''
converts string date to date time object
'''
def get_date_time(str_date):
    if len(str_date) == 0:
        return 'None'
    else:
        dt_format = "%Y-%m-%d %H:%M:%S.%f"
        return datetime.datetime.strptime(str_date, dt_format)

In [5]:
df['SearchDate'] = df['SearchDate'].map(lambda x:get_date_time(x))

In [6]:
df['year'] = df['SearchDate'].map(lambda x: x.year)
df['month'] = df['SearchDate'].map(lambda x: x.month)
df['day'] = df['SearchDate'].map(lambda x: x.day)
df['weekday'] = df['SearchDate'].map(lambda x: x.weekday())
df['hour'] = df['SearchDate'].map(lambda x: x.hour)

# What time of day do most clicks occur?

In [7]:
hour_day_clicks = df.groupby(["weekday", "hour"]).sum()
hour_day_df = hour_day_clicks.reset_index()[['weekday','hour','IsClick']]

In [8]:
# Save DataFrame to csv so we can plot in a heatmap
#hour_day_df.to_csv('heatmap_data.csv')

# How many users enter a search query vs. just filtering by category?

In [23]:
# DataFrame including only users who entered a search query
search_users_df = df[pd.notnull(df['SearchQuery'])]

In [10]:
# Drop unnecessary column
search_users_df.drop('Unnamed: 0',axis=1,inplace=True)

/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


In [24]:
# Count total unique search query users
num_unique_search_users = len(np.unique(search_users_df['UserID']))

In [25]:
# Count total unique users in entire sample
total_unique_users = len(np.unique(df['UserID']))

In [26]:
# Percentage of users who enter a search query
search_user_percentage = np.round(num_unique_search_users / float(total_unique_users) * 100,0)

In [27]:
non_search_user_percentage = 100 - search_user_percentage

In [28]:
search_filter_percent_df = pd.DataFrame({'User_Type':['Search Users','Non-Search Users'],'Percent':[search_user_percentage,non_search_user_percentage]})

In [29]:
# Save as csv and make a pie chart in d3
search_filter_percent_df.to_csv('data/search_filter_percent.csv')

# How does the Ad Click Rate for Search Users compare to the rest?

In [7]:
# Get DF of only unique UserID
distinct_user_df = df.drop_duplicates(subset = 'SearchID')

In [9]:
# Get DataFrame of distinct search query users
distinct_search_user_df = distinct_user_df[pd.notnull(distinct_user_df['SearchQuery'])]

In [57]:
# Get search users CTR
search_users_ctr = round(distinct_search_user_df['IsClick'].mean()*100,1)

In [58]:
search_users_ctr

1.5

In [13]:
# Get DF of users who dont enter a search query
non_search_user_df = distinct_user_df[distinct_user_df['SearchQuery'].isnull()]

In [14]:
# Get DF of users who only use the filter option
filter_users_df = non_search_user_df[pd.notnull(non_search_user_df['SearchParams'])]

In [55]:
filter_users_ctr = round(filter_users_df['IsClick'].mean()*100,2)

In [56]:
filter_users_ctr

0.67

In [29]:
# Get DF of users who don;t enter a search query or use the filter
no_search_filter_user_df = non_search_user_df[non_search_user_df['SearchParams'].isnull()]

In [53]:
no_filter_search_ctr = round(no_search_filter_user_df['IsClick'].mean()*100,2)

In [54]:
no_filter_search_ctr

0.57

In [50]:
# Create DF of ad click rates for the 3 groups
ad_click_rates_df = pd.DataFrame({'User_Type':['Search Users','Filter Users','No Search or Filter'],'Percent':[search_users_ctr,filter_users_ctr,no_filter_search_ctr]})

In [51]:
ad_click_rates_df

,Percent,User_Type
0,1.50,Search Users
1,0.67,Filter Users
2,0.57,No Search or Filter


In [52]:
# Save DF as csv to be plotted
ad_click_rates_df.to_csv('data/ad_click_rates.csv')

# What are the top items users are searching for?

In [10]:
top_searches = distinct_search_user_df.groupby('SearchQuery').size().sort_values(ascending=False)

In [11]:
top_searches

SearchQuery
велосипед                        1297
диван                             609
коляска                           491
туфли                             466
сумка                             450
детские коляски                   393
платье                            374
холодильник                       350
отдам                             343
кроссовки                         327
босоножки                         304
велосипеды бу                     289
ваз                               289
iphone 5s                         274
кровать                           240
детский велосипед                 215
iphone 6                          214
диван бу                          207
самокат                           207
шкаф                              199
прогулочная коляска               194
куртка                            187
комод                             186
iphone                            179
кеды                              173
кресло                            170


In [14]:
top_searches_df = pd.DataFrame({'Item':top_searches.index,'Count':top_searches.values})

In [12]:
russian_translation_dict={
'велосипед':'bike',
'диван':'sofa',
'коляска':'stroller',
'детские коляски':'child stroller',
'туфли':'shoes',
'сумка':'bag'}

In [15]:
top_six_searches_df = top_searches_df.head(6)

In [16]:
top_six_searches_df["Item"].replace(russian_translation_dict, inplace=True)

/Users/jack/anaconda2/lib/python2.7/site-packages/pandas/core/generic.py:3443: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._update_inplace(new_data)


In [17]:
top_six_searches_df

,Count,Item
0,1297,bike
1,609,sofa
2,491,stroller
3,466,shoes
4,450,bag
5,393,child stroller


In [18]:
top_six_searches_df.to_csv('data/top_queries_count.csv')

In [24]:
bike_search_percentage = round(top_six_searches_df['Count'][0] / float(top_six_searches_df['Count'].sum())*100,0)

# Which ads are being shown most, clicked most and why?

In [81]:
# Gets all users who searched for a bike
bicycle_search_users = df[df['SearchQuery'] == 'велосипед']

In [82]:
# Get top shown bike ads
top_bicycle_ads = bicycle_search_users['AdID'].value_counts()

In [105]:
top_five_bicycle_impressions = top_bicycle_ads[:5]

In [106]:
bicycle_ads_df = pd.DataFrame({'AdID':top_five_bicycle_impressions.index,'Impressions':top_five_bicycle_impressions.values})

In [84]:
# Get CTR for all bike ads
bicycle_ads_CTR = bicycle_search_users.groupby('AdID').mean()['IsClick'].sort_values(ascending=False)

In [103]:
# Limit it to the top 6 shown bike ads
top_five_bike_CTR = bicycle_ads_CTR.get([6115044,13485461,36414129,33940616,15719085])

In [104]:
top_five_bike_CTR

AdID
6115044     0.025000
13485461    0.000000
36414129    0.000000
33940616    0.040541
15719085    0.014286
Name: IsClick, dtype: float64

In [110]:
bicycle_ads_df['CTR'] = np.round(top_five_bike_CTR.values*100,2)

In [111]:
bicycle_ads_df.head()

,AdID,Impressions,CTR
0,6115044,200,2.50
1,13485461,139,0.00
2,36414129,93,0.00
3,21649152,77,4.05
4,33940616,74,1.43


In [112]:
bicycle_ads_df.to_csv('top_bike_ads_data.csv')

In [87]:
# Lets see what these top ads say
ad_info = pd.read_csv('AdsInfo.tsv',sep='\t')

In [90]:
bicycle_search_users = bicycle_search_users.merge(ad_info,on='AdID',how='left')

# Is price playing a role in ad CTR?

In [119]:
# Convert RUssian ruble to dollar
bicycle_search_users['USD'] = np.round(bicycle_search_users['Price'] * 0.017,0)

In [123]:
# get all bikes and check price distribution
# Which price point has the most clicks?
bins = np.arange(min(bicycle_search_users['USD']), max(bicycle_search_users['USD']), 50)

In [124]:
bicycle_search_users['price_bin'] = pd.cut(bicycle_search_users['USD'], bins)

In [125]:
bicycle_search_users.col

,Unnamed: 0,SearchID,AdID,Position,ObjectType,HistCTR,IsClick,SearchDate,IPID,UserID,...,weekday,hour,LocationID_y,CategoryID_y,Params,Price,Title,IsContext,USD,price_bin
0,89,57,12340121,7,3,0.003769,0,2015-05-07 19:01:12,316678,2422934,...,3,19,NaN,4,{156:'Горные'},31145,"Горный велосипед Head Troy 2 26"" (2014)",1,529,"(502, 552]"
1,90,57,19750772,1,3,0.025855,0,2015-05-07 19:01:12,316678,2422934,...,3,19,NaN,4,{156:'Дорожные'},10690,Велосипед Forward Volcano 365 RS35/TY18 Black,1,182,"(152, 202]"
2,431,266,3177885,1,3,0.006842,0,2015-05-15 16:20:04,1128821,352247,...,4,16,NaN,4,{156:'Дорожные'},12790,"Велосипед Challenger Desperado Lux 20"" Red",1,217,"(202, 252]"
3,432,266,34434393,7,3,0.001138,0,2015-05-15 16:20:04,1128821,352247,...,4,16,NaN,4,{156:'Горные'},20720,"Женский горный велосипед Head Tacoma 1 26""",1,352,"(302, 352]"
4,774,478,13178651,1,3,0.007233,0,2015-04-26 23:57:43,1010147,3589259,...,6,23,NaN,4,{156:'Дорожные'},15690,"Велосипед Forward Jade 2.0 Disk 17"" White matt",1,267,"(252, 302]"


In [132]:
price_bin_count = bicycle_search_users.groupby('price_bin').size()

In [152]:
# Make the Farmatting of price bins look nicer
def format_bins(text):
    text = text.replace('(', '[').replace(',', ' -')
    return text

In [153]:
format_bins(price_bin_df['price_bin'][0])

'[2 - 52]'

In [134]:
price_bin_clicks = bicycle_search_users.groupby('price_bin')['IsClick'].sum()

In [137]:
price_bin_clicks = price_bin_clicks.fillna(0)

In [ ]:
# Create price_bin DF with Impressions & Clicks

In [145]:
price_bin_df = pd.DataFrame({'price_bin':price_bin_count.index,'count':price_bin_count.values})

In [146]:
price_bin_df['clicks'] = price_bin_clicks.values

In [156]:
price_bin_df['price_bin'] = price_bin_df['price_bin'].apply(lambda x:format_bins(x))

In [ ]:
# Save DF to CSV

In [159]:
price_bin_df.to_csv('price_bin_data.csv')

In [160]:
df.head()

,Unnamed: 0,SearchID,AdID,Position,ObjectType,HistCTR,IsClick,SearchDate,IPID,UserID,IsUserLoggedOn,SearchQuery,LocationID,CategoryID,SearchParams,year,month,day,weekday,hour
0,0,2,11441863,1,3,0.001804,0,2015-05-12 14:21:28,1731568,769304,0,NaN,697,50,NaN,2015,5,12,1,14
1,1,2,22968355,7,3,0.004723,0,2015-05-12 14:21:28,1731568,769304,0,NaN,697,50,NaN,2015,5,12,1,14
2,2,3,212187,7,3,0.029701,0,2015-05-12 07:09:42,793143,640089,0,NaN,1261,12,NaN,2015,5,12,1,7
3,3,3,34084553,1,3,0.004300,0,2015-05-12 07:09:42,793143,640089,0,NaN,1261,12,NaN,2015,5,12,1,7
4,4,4,20653823,1,3,0.003049,0,2015-05-10 18:11:01,898705,3573776,0,NaN,3960,22,"{83:'Обувь', 175:'Женская одежда', 88:'38'}",2015,5,10,6,18
